In [1]:
import pandas as pd
from error_config import error_map

In [21]:
df_transformer = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/scored/dialogue_aggrefact_scored.csv')

# df_llm = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/llm_annotations/sample/sample_annotations_sanjana.csv')
# df_llm = df_llm.sort_values('uuid').drop_duplicates(['docid'], keep='last')

# len(df_llm)





In [3]:
len(

,Unnamed: 0.1,Unnamed: 0,DocID,Dialogue,Model,Summary,w/ Error,CorefE,CorefE_text,CircE,...,PredE_text,SubjObjE,SubjObjE_text,OtherE,OtherE_text,LinkE,LinkE_text,origin,SummaC-ZS_score,SummaC-Conv_score
0,0,0,13809941,Thelma: i dont have anything to wear\nLouisa: ...,human_ref,Louisa will lend Thelma her red velvet dress.,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.608673,0.224066
1,1,1,13809941,Thelma: i dont have anything to wear\nLouisa: ...,bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.271927,0.335007
2,2,2,13809941,Thelma: i dont have anything to wear\nLouisa: ...,mv-bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.271927,0.335007
3,3,3,13809941,Thelma: i dont have anything to wear\nLouisa: ...,co-ref bart large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.090210,0.329921
4,4,4,13809941,Thelma: i dont have anything to wear\nLouisa: ...,condigsum bart large,Louisa will bring Thelma her red velvet dress.,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.374329,0.224066


In [4]:
# df_transformer['Dialogue'].values

In [5]:



qafact_kwargs = {"cuda_device": 0, "use_lerc_quip": True, \
        "verbose": True, "generation_batch_size": 32, \
        "answering_batch_size": 32, "lerc_batch_size": 8}

qafact_model_folder = "/home/sanjana/factual_evaluation_source_based/experiments/QAFactEval/models" # path to models downloaded with download_models.sh


class Evaluation():
    def __init__(self):
        return


    def get_questeval_scores(self, df, source_key, summary_key):
        from questeval.questeval_metric import QuestEval
        questeval = QuestEval(no_cuda=True)
        sources = list(df[source_key].values)
        summaries = list(df[summary_key].values)
        score = questeval.corpus_questeval(
            hypothesis=summaries, 
            sources=sources,
        )
        df['QuestEval_score'] = score['ex_level_scores']
        return df

    def get_qafacteval_scores(self, df, source_key, summary_key):
        from qafacteval import QAFactEval
        qafacteval = QAFactEval(
            lerc_quip_path=f"{qafact_model_folder}/quip-512-mocha",
            generation_model_path=f"{qafact_model_folder}/generation/model.tar.gz",
            answering_model_dir=f"{qafact_model_folder}/answering",
            lerc_model_path=f"{qafact_model_folder}/lerc/model.tar.gz",
            lerc_pretrained_model_path=f"{qafact_model_folder}/lerc/pretraining.tar.gz",
            **qafact_kwargs
        )
        sources = list(df[source_key].values)
        summaries = list(df[summary_key].values)
        summaries = [[summ] for summ in summaries]
        results = qafacteval.score_batch_qafacteval(sources, summaries, return_qa_pairs=True)
        scores = [ each[0]['qa-eval']['lerc_quip'] for each in results]
        df['QAFactEval_score'] = scores
        return df

    def get_summac_scores(self, df, source_key, summary_key):
        from summac.model_summac import SummaCZS, SummaCConv
        model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda") # If you have a GPU: switch to: device="cuda"
        model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cuda", start_file="default", agg="mean")

        sources = list(df[source_key].values)
        summaries = list(df[summary_key].values)
        score_zs = model_zs.score(sources, summaries)
        score_conv = model_conv.score(sources, summaries)
        df['SummaC-ZS_score']= score_zs['scores']
        df['SummaC-Conv_score'] = score_conv['scores']
        return df

        
    

        
        

In [6]:
EvalMetrics = Evaluation()


In [7]:
df_transformer = EvalMetrics.get_questeval_scores(df_transformer, 'Dialogue', 'Summary')

2023-09-29 12:34:18.312317: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 12:34:20.065510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a Ber

In [8]:
df_transformer.head()

,Unnamed: 0.1,Unnamed: 0,DocID,Dialogue,Model,Summary,w/ Error,CorefE,CorefE_text,CircE,...,SubjObjE,SubjObjE_text,OtherE,OtherE_text,LinkE,LinkE_text,origin,SummaC-ZS_score,SummaC-Conv_score,QuestEval_score
0,0,0,13809941,Thelma: i dont have anything to wear\nLouisa: ...,human_ref,Louisa will lend Thelma her red velvet dress.,0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.608673,0.224066,0.642865
1,1,1,13809941,Thelma: i dont have anything to wear\nLouisa: ...,bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.271927,0.335007,0.513751
2,2,2,13809941,Thelma: i dont have anything to wear\nLouisa: ...,mv-bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.271927,0.335007,0.513751
3,3,3,13809941,Thelma: i dont have anything to wear\nLouisa: ...,co-ref bart large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.090210,0.329921,0.403632
4,4,4,13809941,Thelma: i dont have anything to wear\nLouisa: ...,condigsum bart large,Louisa will bring Thelma her red velvet dress.,0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.374329,0.224066,0.651178


In [9]:
df_transformer.to_csv('/home/sanjana/factual_evaluation_source_based/datasets/scored/dialogue_aggrefact_scored.csv')

In [10]:
from sklearn.metrics import roc_auc_score

def calculate_auc(y, predictions):
    print(roc_auc_score(y, predictions))

In [18]:
y = df_transformer['w/ Error'].values
predictions = [1 - each for each in df_transformer['SummaC-ZS_score'].values]
calculate_auc(y, predictions)

0.6206089531376444


In [19]:
y = df_transformer['w/ Error'].values
predictions = [1- each for each in df_transformer['SummaC-Conv_score'].values]
calculate_auc(y, predictions)

0.4845313304642095


In [20]:
y = df_transformer['w/ Error'].values
predictions = [1 - each for each in df_transformer['QuestEval_score'].values]
calculate_auc(y, predictions)

0.6349557216949901
